In [14]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt

def importarPQR():
    datos = pd.read_excel('FP.xlsx', header=2, nrows=5)
    return datos

datosExcel = importarPQR()


colaMaquina1 = np.array([])
colaMaquina2 = np.array([])
colaMaquina3 = np.array([])
colaMaquina4 = np.array([])
colaMaquina5 = np.array([])

maquinas = np.array([colaMaquina1, colaMaquina2, colaMaquina3, colaMaquina4, colaMaquina5])


def iniciarCromosoma():
    return np.array([10, 3, 7, 4, 5])


def asignarRestantes():
    global datosExcel
    global maquinas
    faltar = [1, 2, 6, 8, 9]
    for trabajo in faltar:
        variableExcel = 'J' + str(trabajo)
        maquina = datosExcel[variableExcel][0]
        np.insert(maquinas[maquina-1], trabajo)



0    1
1    2
2    3
3    4
4    5
Name: J1, dtype: int64